In [33]:
# Created by: Michael Cullen
# 08/10/2024
# https://zenodo.org/records/10457828

In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('vehicles.csv')

df.head()

In [ ]:
# Drop not needed columns
df = df.drop(columns=['region_url','description','url','id','posting_date','lat','long','VIN','image_url','county','state','region','size','model','paint_color','title_status','type'], errors='ignore')

df = df.iloc[30:]
df = df.fillna('unknown')


df['fuel'] = df['fuel'].replace({
    'other': 'diesel',  # Replace 'other' with 'diesel'
    'gas': 'petrol'     # Replace 'gas' with 'petrol'
})

# Replace 'other' in the 'transmission' column
df['transmission'] = df['transmission'].replace({
    'other': 'manual'   # Replace 'other' with 'manual'
})


df.head(20)


,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive
30,30990,2017.0,toyota,good,8 cylinders,petrol,41124.0,manual,unknown
31,15000,2013.0,ford,excellent,6 cylinders,petrol,128000.0,automatic,rwd
32,27990,2012.0,gmc,good,8 cylinders,petrol,68696.0,manual,4wd
33,34590,2016.0,chevrolet,good,6 cylinders,petrol,29499.0,manual,4wd
34,35000,2019.0,toyota,excellent,6 cylinders,petrol,43000.0,automatic,4wd
35,29990,2016.0,chevrolet,good,6 cylinders,petrol,17302.0,manual,4wd
36,38590,2011.0,chevrolet,good,8 cylinders,petrol,30237.0,manual,rwd
37,4500,1992.0,jeep,excellent,6 cylinders,petrol,192000.0,automatic,4wd
38,32990,2017.0,jeep,good,6 cylinders,petrol,30041.0,manual,4wd
39,24590,2017.0,chevrolet,good,6 cylinders,petrol,40784.0,manual,unknown


In [ ]:
# Convert odometer to numeric
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')

# Define the function to categorize odometer values
def categorize_odometer(odometer):  # 1 = missing, 2 = 0-20k, 3 = 20-50k, 4 = 50-100k, 5 = 100-150k, 6 = 150-200k, 7 = 200k+
    if odometer is None or odometer < 0 or pd.isna(odometer):
        return 1  # Missing
    elif odometer <= 20000:
        return 2  # 0-20k
    elif odometer <= 50000:
        return 3  # 20-50k
    elif odometer <= 100000:
        return 4  # 50-100k
    elif odometer <= 150000:
        return 5  # 100-150k
    elif odometer <= 200000:
        return 6  # 150-200k
    else:
        return 7  # 200k+
    
    # Define function to categorize year
def categorize_year(year):  # 1 = very old, 2 = old, 3 = modern, 4 = new
    if year is None or pd.isna(year):
        return 1  # Missing
    elif year < 1990:
        return 1  # Very old
    elif year < 2005:
        return 2  # Old
    elif year < 2015:
        return 3  # Modern
    else:
        return 4  # New

# Define function to categorize price
def categorize_price(price):  # 1 = low, 2 = medium, 3 = high, 4 = premium
    if price is None or pd.isna(price):
        return 1  # Missing
    elif price < 5000:
        return 1  # Low
    elif price < 20000:
        return 2  # Medium
    elif price < 40000:
        return 3  # High
    else:
        return 4  # Premium

# Apply the function to create the 'odometer_group' column
df['odometer_group'] = df['odometer'].apply(categorize_odometer)
df['year_group'] = df['year'].apply(categorize_year)
df['price_group'] = df['price'].apply(categorize_price)

# Drop the original 'odometer' column
df = df.drop(columns=['odometer', 'year', 'price'], errors='ignore')

# Show the first 10 rows
df.head(10)


KeyError: 'odometer'

In [ ]:
print(df.shape[0])

426850


In [ ]:
# Define features (X) and target (y)
X = df.drop(columns=['price'])
y = df['price']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=50, max_depth=5)  # Use fewer trees
rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)

MemoryError: Unable to allocate 12.8 GiB for an array with shape (128055, 13462) and data type float64

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)